In [9]:
!pip uninstall openai
!pip install openai

Found existing installation: openai 0.28.0
Uninstalling openai-0.28.0:
  Would remove:
    /usr/local/bin/openai
    /usr/local/lib/python3.10/dist-packages/openai-0.28.0.dist-info/*
    /usr/local/lib/python3.10/dist-packages/openai/*
Proceed (Y/n)? y
  Successfully uninstalled openai-0.28.0
  Using cached openai-1.35.3-py3-none-any.whl (327 kB)


In [6]:
from google.colab import drive

drive.mount('/content/drive')
%cd /content/drive/MyDrive/recipe

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/recipe


In [9]:
import os
import pickle
import pandas as pd
import torch
from transformers import BertTokenizer, BertModel
from sklearn.metrics.pairwise import cosine_similarity
import openai
import os

# Set OpenAI API key
os.environ["OPENAI_API_KEY"] = ''
openai.api_key = os.environ["OPENAI_API_KEY"]

# Parameters
MODEL_NAME = "gpt-4"
TEMPERATURE = 0

PERSONA = """
다음 절차를 따라 사용자의 질문에 답변.

첫째, 사용자가 제공한 조건에 따라 데이터에서 해당 레시피들을 찾아서 해당 내용을 구체적으로 답변.
둘째, 만약 제공한 조건에 따른 데이터가 많아도 하나만 출력.
셋째, 데이터에 적합한 내용이 없으면 대답하지 않음.
넷째, 각 레시피를 다음과 같이 출력.

  요리 이름:
  종류:
  필요한 재료:
  난이도:
  요리 시간:

"""

def load_model_and_embeddings(file_path='model_embeddings.pkl'):
    """Load BERT model and embeddings from a pickle file."""
    with open(file_path, 'rb') as f:
        model, embeddings = pickle.load(f)
    print(f"Model and embeddings loaded from {file_path}")
    return model, embeddings

def get_bert_embeddings(text_list, tokenizer, model):
    """Generate BERT embeddings for a list of texts."""
    inputs = tokenizer(text_list, return_tensors='pt', padding=True, truncation=True, max_length=512)
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.last_hidden_state.mean(dim=1).numpy()

def recommend_recipe(user_input, model, data_embeddings, tokenizer, file_path='Recipe_Info.txt'):
    """Recommend recipes based on user input using BERT embeddings."""
    data = pd.read_csv(file_path, delimiter='\t', header=None, names=["combined_features"])
    user_embedding = get_bert_embeddings([user_input], tokenizer, model)
    sim_scores = cosine_similarity(user_embedding, data_embeddings).flatten()
    sim_indices = sim_scores.argsort()[-5:][::-1]
    recommendations = [(data.iloc[i]['combined_features'], sim_scores[i]) for i in sim_indices]
    response = ""
    for recipe, score in recommendations:
        response += f"Similarity Score: {score}\n"
        response += f"{recipe}\n"
        response += "-" * 20 + "\n"
    return response

def generate_response(persona, user_input, recommendation_response):
    response = openai.ChatCompletion.create(
        model="gpt-4",
        messages=[
            {"role": "system", "content": persona},
            {"role": "user", "content": user_input},
            {"role": "assistant", "content": recommendation_response}],
        temperature=0
    )
    message = response.choices[0].message.content
    return message

def main():
    """Main function to run the recipe recommendation system."""
    # Load the BERT model and embeddings
    loaded_model, loaded_embeddings = load_model_and_embeddings('model_embeddings.pkl')

    # Initialize BERT tokenizer
    tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

    print("Chatbot is now running. Type 'exit' to quit.")
    while True:
        user_input = input("User: ")
        if user_input.lower() == 'exit':
            break
        recommendation_response = recommend_recipe(user_input, loaded_model, loaded_embeddings, tokenizer)
        chatbot_response = generate_response(PERSONA, user_input, recommendation_response)
        print(f"Chatbot: {chatbot_response}")

if __name__ == "__main__":
    main()


Model and embeddings loaded from model_embeddings.pkl
Chatbot is now running. Type 'exit' to quit.
User: 고기
Chatbot: 요리 이름: 찜닭
종류: 메인반찬
필요한 재료: 닭 중간크기 1마리, 청양고추 5개, 건고추 5개
난이도: 초급
요리 시간: 60분이내


KeyboardInterrupt: Interrupted by user

In [1]:
pip install -U openai==0.28